In [1]:
#Import pandas
from __future__ import print_function

import pandas as pd
pd.__version__

'1.1.5'

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from google.colab import files

[Load data](https://data.ny.gov/Energy-Environment/Oil-Gas-Other-Regulated-Wells-Beginning-1860/szye-wmt3)

In [3]:
df_raw = pd.read_csv("wells_1860.csv")
#Remove spaces in column names
df_raw.columns = df_raw.columns.str.replace(' ', '')
df_raw.columns = df_raw.columns.str.replace(',', '')
df_raw.head(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,APIWellNumber,CountyCode,APIHoleNumber,Sidetrack,Completion,WellName,CompanyName,OperatorNumber,WellType,MapSymbol,WellStatus,StatusDate,PermitApplicationDate,PermitIssuedDate,DateSpudded,DateofTotalDepth,DateWellCompleted,DateWellPlugged,DateWellConfidentialityEnds,ConfidentialityCode,Town,Quad,QuadSection,ProducingField,ProducingFormation,FinancialSecurity,Slant,County,Region,StateLease,ProposedDepthft,SurfaceLongitude,SurfaceLatitude,BottomHoleLongitude,BottomHoleLatitude,TrueVerticalDepthft,MeasuredDepthft,Kickoffft,DrilledDepthft,Elevationft,OriginalWellType,PermitFee,ObjectiveFormation,DepthFee,Spacing,SpacingAcres,Integration,HearingDate,DateLastModified,DECDatabaseLink,Location1
0,31003000850000,3,85,0,0,Woodruff Fanny Est,Kerr & Kaufman,9271,GD,GWP,PA,NaN,NaN,NaN,NaN,NaN,04/13/1933,NaN,NaN,Pre-1989 Well (N/A),NaN,NaN,NaN,NaN,NaN,False,Vertical,Statewide,9.0,NaN,0.0,NaN,NaN,NaN,NaN,2973.0,2973.0,0.0,2973.0,1644.0,NL,0.0,NaN,0.0,NaN,NaN,NaN,NaN,10/12/1995 12:00:00 AM +0000,http://www.dec.ny.gov/cfmx/extapps/GasOil/sear...,NaN
1,31003003550000,3,355,0,0,Hoyt E C 1,Jackles et al,9262,DH,DP,PA,NaN,NaN,NaN,NaN,NaN,10/10/1940,NaN,NaN,Pre-1989 Well (N/A),Allen,Fillmore,F,Allen,NaN,False,Vertical,Allegany,9.0,NaN,0.0,-78.01308,42.41949,-78.01308,42.41949,2941.0,2941.0,0.0,2941.0,1641.0,NL,0.0,NaN,0.0,NaN,NaN,NaN,NaN,01/25/1999 12:00:00 AM +0000,http://www.dec.ny.gov/cfmx/extapps/GasOil/sear...,"(42.41949, -78.01308)"
2,31003045950000,3,4595,0,0,Old #2 Lot 79 Bradley-Thornton,Bradley Producing Company,9673,NL,O,VP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pre-1989 Well (N/A),Alma,NaN,NaN,NaN,NaN,False,Vertical,Allegany,9.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NL,0.0,NaN,0.0,NaN,NaN,NaN,NaN,05/01/2014 12:00:00 AM +0000,http://www.dec.ny.gov/cfmx/extapps/GasOil/sear...,NaN


In [4]:
#Filtering to keep only oil and gas wells. Confidential or dry holes are also omitted
#Dry hole: well found to be incapable of producing either oil or gas in sufficient quantities to justify completion as an oil or gas well.
#https://www.eia.gov/tools/glossary/index.php?id=dry
df = df_raw
df = df[~df["MapSymbol"].isin(["CON"])]
df = df[~df["MapSymbol"].isin(["DH"])]
df = df[~df["MapSymbol"].isin(["SM"])]
df = df[~df["MapSymbol"].isin(["SMP"])]
df = df[~df["MapSymbol"].isin(["O"])]
df = df[~df["MapSymbol"].isin(["OP"])]
len(df)

31185

In [5]:
#Filtering to only active wells
active = df[df["WellStatus"].isin(["AC"])]
#Counting number of active wells
len(active)

11519

In [6]:
#Download CSV of all active wells – remove # in below script to run the download

#active.to_csv('active_all.csv') 
#files.download('active_all.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Counting number of active wells in each county
active_cty = active.County.value_counts().sort_values(ascending=False).rename_axis('County').reset_index(name='Ct')
active_cty

,County,Ct
0,Chautauqua,4075
1,Cattaraugus,2607
2,Erie,1420
3,Allegany,1209
4,Genesee,485
5,Steuben,394
6,Wyoming,326
7,Cayuga,296
8,Seneca,221
9,Livingston,134


In [ ]:
#Download CSV of number of active wells in each county – remove # in below script to run the download

#active_cty.to_csv('active.csv') 
#files.download('active.csv')

Plugged and abandoned wells in each county

In [ ]:
#Filtering to only the plugged and abandoned wells
plug_abdn = df[df["WellStatus"].isin(["PA"])]
#Counting number of plugged and abandoned wells
len(plug_abdn)

13884

In [ ]:
#Counting number of plugged and abandoned wells in each county
plug_abdn_ct = plug_abdn.groupby(['County']).size().reset_index(name='Count')
plug_abdn_ct.sort_values('Count', ascending=False)

,County,Count
0,Allegany,6443
2,Cattaraugus,3632
10,Erie,1105
4,Chautauqua,860
34,Steuben,686
42,Wyoming,210
12,Genesee,195
24,Ontario,75
16,Livingston,74
3,Cayuga,60


In [ ]:
#Download CSV of number of plugged and abandoned wells in each county – remove # in below script to run the download

#plug_abdn_ct.to_csv('plug_abdn.csv') 
#files.download('plug_abdn.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Time between well completion and date plugged

In [ ]:
#Converting dates to Pandas date format
df['DateWellCompleted'] = pd.DatetimeIndex(df['DateWellCompleted'])
df['DateWellPlugged'] = pd.DatetimeIndex(df['DateWellPlugged'])

In [ ]:
#Make new variable subtracting the time between plugged and completed
df['WellTime'] = df['DateWellPlugged'] - df['DateWellCompleted']

In [ ]:
#Filtering to remove NAs that result from NAs in either DateWellPlugged or DateWellCompleted variable
df_nona = df[~df['WellTime'].isna()]

In [ ]:
#Converting WellTime value (i.e, 5 days) to string to remove the word "days," so I am left with just the number 5 to convert to numeric form after
df_nona['WellTime'] = df_nona['WellTime'].astype(str)
df_nona['WellTime'] = df_nona['WellTime'].str.replace(' days', '')

In [ ]:
#Convert to float
df_nona['WellTime'] = df_nona['WellTime'].astype(float)

In [ ]:
#Filtering to only include positive times; 52 wells were omitted because they were recorded as plugged before completed.
df_pos_time = df_nona[df_nona['WellTime']>0]

In [ ]:
#Looking at summary stats for the time in days between well completion and well plugging
df_pos_time.WellTime.describe()

count     3434.00000
mean     11007.20297
std       9572.08331
min          1.00000
25%       3118.00000
50%       8704.00000
75%      16634.75000
max      47326.00000
Name: WellTime, dtype: float64

In [ ]:
#Taking a closer look at the outlier, maximum value
df_pos_time[df_pos_time['WellTime']==47326]

,APIWellNumber,CountyCode,APIHoleNumber,Sidetrack,Completion,WellName,CompanyName,OperatorNumber,WellType,MapSymbol,WellStatus,StatusDate,PermitApplicationDate,PermitIssuedDate,DateSpudded,DateofTotalDepth,DateWellCompleted,DateWellPlugged,DateWellConfidentialityEnds,ConfidentialityCode,Town,Quad,QuadSection,ProducingField,ProducingFormation,FinancialSecurity,Slant,County,Region,StateLease,ProposedDepthft,SurfaceLongitude,SurfaceLatitude,BottomHoleLongitude,BottomHoleLatitude,TrueVerticalDepthft,MeasuredDepthft,Kickoffft,DrilledDepthft,Elevationft,OriginalWellType,PermitFee,ObjectiveFormation,DepthFee,Spacing,SpacingAcres,Integration,HearingDate,DateLastModified,DECDatabaseLink,Location1,WellTime
42380,31075024370000,75,2437,0,0,Root Charles Beldock 0,Sandy Creek Oil & Gas,9642,GD,GWP,PA,12/18/2018,NaN,NaN,NaN,NaN,1890-01-01,2019-07-30,NaN,Pre-1989 Well (N/A),Sandy Creek,Sandy Creek,G,Sandy Creek,Trenton,False,Vertical,Oswego,7.0,NaN,0.0,-76.089137,43.634459,-76.089137,43.634459,1265.0,1265.0,0.0,1265.0,470.0,NL,0.0,NaN,0.0,NaN,NaN,NaN,NaN,09/18/2019 12:00:00 AM +0000,http://www.dec.ny.gov/cfmx/extapps/GasOil/sear...,"(43.634459, -76.089137)",47326.0


In [ ]:
#Creating a new variable with WellTime as years instead of days
df_pos_time['WellTime_Yr'] = df_pos_time['WellTime']/365

In [ ]:
#Looking at summary stats for the time in years between well completion and well plugging
df_pos_time.WellTime_Yr.describe()

count    3434.000000
mean       30.156720
std        26.224886
min         0.002740
25%         8.542466
50%        23.846575
75%        45.574658
max       129.660274
Name: WellTime_Yr, dtype: float64

In [ ]:
#Download CSV of all wells with positive times – remove # from below script to run the download

#df_pos_time.to_csv('timespan.csv')
#files.download('timespan.csv')

In [ ]:
#Find means and other summary statistics for years between well completion and plugging in each county
df_cty_summ = df_pos_time.groupby(['County']).WellTime_Yr.describe().reset_index()
df_cty_summ.sort_values('mean', ascending=False)

,County,count,mean,std,min,25%,50%,75%,max
16,Oneida,7.0,55.947945,38.164230,0.041096,38.850685,78.260274,78.487671,78.657534
13,Monroe,9.0,46.063927,13.299624,23.123288,47.561644,47.947945,55.493151,57.367123
21,Oswego,9.0,39.059970,52.535876,0.005479,4.273973,9.884932,85.939726,129.660274
0,Allegany,698.0,36.801303,26.759292,0.002740,16.051370,26.932877,60.939041,122.389041
2,Cattaraugus,1130.0,36.498778,24.676284,0.002740,20.707534,33.636986,48.180137,115.901370
18,Ontario,40.0,33.665959,29.493983,0.038356,5.776712,28.789041,53.154795,104.778082
25,Steuben,170.0,32.856906,29.506957,0.002740,7.796575,24.006849,54.506849,103.928767
10,Genesee,137.0,31.915508,26.646796,0.002740,10.986301,24.789041,46.139726,114.942466
23,Schuyler,15.0,29.377900,28.959890,0.153425,6.283562,10.602740,47.558904,86.780822
9,Erie,389.0,26.696327,27.602028,0.002740,4.430137,15.542466,39.506849,120.504110


In [ ]:
#df_cty_summ.to_csv('timespan_summ.csv')
#files.download('timespan_summ.csv')